In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.svm import SVC
from sklearn.datasets import make_blobs, make_circles
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, train_test_split, GridSearchCV
from scipy.stats import multivariate_normal

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
plt.style.use('ggplot')

ImportError: cannot import name 'DecisionBoundaryDisplay' from 'sklearn.inspection' (/Users/leonardohannas1998/opt/anaconda3/lib/python3.9/site-packages/sklearn/inspection/__init__.py)

# TP: Support Vector Machine (SVM)

## Students:
* **Juan Esteban Rios Gallego**
* **Leonardo Hannas de Carvalho Santos**

## Preliminary questions

**1)** Show that the primal problem solved by the SVM can be rewritten as follows :

$$ \underset{\mathbf{w} \in \mathcal{H}, w_0 \in \mathbb{R}}{argmin} \left( \frac{1}{2}||\mathbf{w}||^2 + C \sum_{i=1}^n [ 1 - y_i ( \langle \mathbf{w}, \Phi(\mathbf{x_i}) \rangle + w_0 )]_+ \right) $$


<div class='alert alert-block alert-warning'>
            Answer:</div>

We first consider the equation of the hyperplane in the feature space $\mathcal{H}$, which is given by:
$$ \langle \mathbf{w}, \Phi(\mathbf{x}) \rangle + w_0 = 0 $$

The geometrical margin of the hyperplane is given by:
$$ \rho(\mathbf{w}) = \frac{1}{||\mathbf{w}||} $$

Moreover, the classification problem is given by, $\forall i = \{1, \cdots, n\}$:
$$
\left\{\begin{matrix}
y_{i} = 1, \text{ if } \langle \mathbf{w}, \Phi(\mathbf{x}) \rangle + w_0 \geq 1\\ 
y_{i} = -1, \text{ if } \langle \mathbf{w}, \Phi(\mathbf{x}) \rangle + w_0 \leq -1
\end{matrix}\right.
$$

which can be written as, $\forall i = \{1, \cdots, n\}$:
$$
y_{i} (\langle \mathbf{w}, \Phi(\mathbf{x}) \rangle + w_0) \geq 1 \iff 
1 - y_{i} (\langle \mathbf{w}, \Phi(\mathbf{x}) \rangle + w_0) \leq 0
$$

Since the goal is to maximize the margin, we can write the optimization problem as follows:
$$
\underset{\mathbf{w}}{min}\frac{1}{2}||\mathbf{w}||^2 \text{ subject to } 1 - y_{i} (\langle \mathbf{w}, \Phi(\mathbf{x}) \rangle + w_0) \leq 0 \text{, } \forall i = \{1, \cdots, n\}
$$

Now, we can introduce the slack variables $\xi_i \geq 0$ to allow some points to be inside the margin or misclassified. The optimization problem becomes:
$$
\underset{\mathbf{w}, \xi}{min}\frac{1}{2}||\mathbf{w}||^2 + C \sum_{i=1}^n \xi_i \text{ subject to } 1 - y_{i} (\langle \mathbf{w}, \Phi(\mathbf{x}) \rangle + w_0) \leq \xi_i \text{, } \forall i = \{1, \cdots, n\}
$$
where $C$ is a regularization parameter that controls the trade-off between the margin and the slack variables. The larger the value of $C$, the more the model will try to minimize the slack variables, and the smaller the value of $C$, the more the model will try to maximize the margin.

By observing the constraints of the slack variables, $\forall i = \{1, \cdots, n\}$:
$$
\begin{cases}
    \quad \xi_i \ge 0 \\
    \quad \xi_i \ge 1 - y_i(\langle \mathbf{w}, \Phi(\mathbf{x}_i) \rangle + w_0)
\end{cases}
$$
they can be written as, $\forall i = \{1, \cdots, n\}$:
$$ \xi_i \geq max(1 - y_i(\langle \mathbf{w}, \Phi(\mathbf{x}_i) \rangle + w_0), 0) = [1 - y_i(\langle \mathbf{w}, \Phi(\mathbf{x}_i) \rangle + w_0)]_+ $$

Finally, the primal problem, considering the slack variables, is done by:
$$
\underset{\mathbf{w} \in \mathcal{H}, w_0 \in \mathbb{R}}{min} \left( \frac{1}{2}||\mathbf{w}||^2 + C \sum_{i=1}^n [ 1 - y_i ( \langle \mathbf{w}, \Phi(\mathbf{x_i}) \rangle + w_0 )]_+ \right)
$$
or, equivalently,
$$
\mathbf{w^{\star}} \in \underset{\mathbf{w} \in \mathcal{H}, w_0 \in \mathbb{R}}{argmin} \left( \frac{1}{2}||\mathbf{w}||^2 + C \sum_{i=1}^n [ 1 - y_i ( \langle \mathbf{w}, \Phi(\mathbf{x_i}) \rangle + w_0 )]_+ \right)
$$

**2)** Explain the sentence : "an SVM minimizes the classification error using a convex upper bound". The function $x \rightarrow [1 - x]_+ = \text{max}(0, 1-x)$ is called *Hinge* (*charnière* en français). Explain the difference between the pivotal loss and the loss of binary classification.

<div class='alert alert-block alert-warning'>
            Answer:</div>

* **Explanation of the sentence:** 

Convex optimization problems are those in which the set of possible solutions forms a convex set, and the objective function (Hinge Loss) to be minimized is a convex function. This characteristic makes the solution easier to find and more reliable, given that we can apply well known algorithms, namely gradient descent, stochastic gradient descent, Newton's method, etc. This is not the case if the objective function is non-convex, as it is the case of the 0-1 loss, though.

* **Difference between the pivotal loss and the loss of binary classification:** 

In this question, we are considering that the pivotal loss is a synonym for the hinge loss of binary classification.
While the 0-1 loss directly measures classification accuracy but is non-convex and difficult to optimize, the hinge loss offers a convex and continuous alternative that still enforces correct classification with a margin. The optimization of the hinge loss thus allows SVMs to benefit from the mathematical advantages of convex optimization, leading to a robust classifier that maximizes the margin between the classes, which is indicative of a model that generalizes well to unseen data.

## Example of using the SVC class from scikit-learn

In [ ]:
#  Data Generation

def rand_gauss(n=100, mu=[1, 1], sigmas=[0.1, 0.1]):
    """ Sample n points from a Gaussian variable with center mu,
    and std deviation sigma
    """
    d = len(mu)
    res = np.random.randn(n, d)
    return np.array(res * sigmas + mu)


def rand_bi_gauss(n1=100, n2=100, mu1=[1, 1], mu2=[-1, -1], sigmas1=[0.1, 0.1],
                  sigmas2=[0.1, 0.1]):
    """ Sample n1 and n2 points from two Gaussian variables centered in mu1,
    mu2, with respective std deviations sigma1 and sigma2
    """
    ex1 = rand_gauss(n1, mu1, sigmas1)
    ex2 = rand_gauss(n2, mu2, sigmas2)
    y = np.hstack([np.ones(n1), -1 * np.ones(n2)])
    X = np.vstack([ex1, ex2])
    ind = np.random.permutation(n1 + n2)
    return X[ind, :], y[ind]

In [ ]:
# This is an example of using SVC for data generated with the above function
n1 = 20
n2 = 20
mu1 = [1., 1.]
mu2 = [-1., -1.]
sigma1 = [0.9, 0.9]
sigma2 = [0.9, 0.9]
X1, y1 = rand_bi_gauss(n1, n2, mu1, mu2, sigma1, sigma2)

X_train = X1[::2]
Y_train = y1[::2].astype(int)
X_test = X1[1::2]
Y_test = y1[1::2].astype(int)

# fit the model with linear kernel
clf = SVC(kernel='linear')
clf.fit(X_train, Y_train)

# predict labels for the test data base
y_pred = clf.predict(X_test)

# check your score
score_train = clf.score(X_train, Y_train)
score_test = clf.score(X_test, Y_test)
print('Training score : %s' % score_train)
print('Testing score : %s' % score_test)

# display the points
plt.figure(1, figsize=(5, 5))
ax = plt.gca()
DecisionBoundaryDisplay.from_estimator(
    clf,
    X1,
    plot_method="contour",
    colors="k",
    levels=[0],
    alpha=0.5,
    ax=ax,
)
ax.scatter(X_train[:, 0], X_train[:, 1], c=Y_train)
ax.scatter(X_test[:, 0], X_test[:, 1], c=Y_test, cmap=plt.cm.Paired)
plt.title('First data set')
plt.axis('equal')
plt.show()

## Linear SVM

**3)** Draw a i.i.d. sample from a mixture of two Gaussian distributions : each class is a Gaussian with specific parameters. This time, use the function ```make_blobs``` available in ```sklearn.datasets``` library. Reserve 75% of the data for training and 25% for the test data.

<div class='alert alert-block alert-info'>
            Code:</div>

In [ ]:
# Create two centers for the two distributions 
centers = [[1, 1], [-2, -2]]

# Use make_blobs to generate the two dimensions points from the two centers
X, y = make_blobs(n_samples=100, centers=centers, n_features=2 ,random_state=0)

# Plot the points 
pos = np.where(y == 1)[0]
neg = np.where(y == 0)[0]
plt.scatter(X[pos,0], X[pos,1], c='r')
plt.scatter(X[neg,0], X[neg,1], c='b')
plt.axis('equal')
plt.show()

In [ ]:
# Use sklearn's train_test_split to divide up data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

**4)** Since the probability distributions are known, numerically estimate the Bayes risk. 

<div class='alert alert-block alert-info'>
            Code:</div>

In [ ]:
from scipy.stats import multivariate_normal
# Do a Monte-Carlo estimation of the Bayes Risk (given the gaussian distributions you used to generate data)
# You need to use the imported class from scipy to generate a large number of samples which you will use 
# to approximate the integral of the Bayes risk
n_mc = X.shape[0] # Repeat this n_mc times - enough to approximate 
expectation = 0
for i in range(n_mc):
    rand = np.random.choice(y) # Random binary choice: will the point be generated from the first or second gaussian ? 
    if rand == 0:
        # First case: y = 0
        x = np.random.multivariate_normal(mean=np.mean(X[pos,:], axis=0), cov=np.cov(X[pos,:].T))
    else:
        # Second case: y = 1
        x = np.random.multivariate_normal(mean=np.mean(X[neg,:], axis=0), cov=np.cov(X[neg,:].T))

    # You have to compute the conditional posterior probability of x given the 2 gaussians 
    # Use the multivariate_normal.pdf() method !    
    p1 = multivariate_normal.pdf(x, mean=np.mean(X[pos,:], axis=0), cov=np.cov(X[pos,:].T))
    p2 = multivariate_normal.pdf(x, mean=np.mean(X[neg,:], axis=0), cov=np.cov(X[neg,:].T))
    # Compute the risk from these and add it to the total
    risk = min(p1, p2)
    risk = 1 - risk if rand == 0 else risk
    expectation += min(p1/(p1+p2), p2/(p1+p2))

expectation /= n_mc

print(f'Estimated Bayes risk: {np.around(expectation, 3)}')
print(f'Estimated Bayes accuracy: {1 - np.around(expectation, 3)}')

**5)** Draw the decision boundary $H$ induced by SVM as well as the hyperplanes $H_1$ and $H_{−1}$. Vary the parameter C to see its impact on the number of support vectors. We can use the code in the following example: https://scikit-learn.org/stable/auto_examples/svm/plot_separating_hyperplane.html.

<div class='alert alert-block alert-info'>
            Code:</div>

In [ ]:
# Create a linear SVM and train it on the training data
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)
print(f'Train/Test scores: {clf.score(X_train, y_train)}/{clf.score(X_test, y_test)}')

# Plot the data
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, s=30, cmap=plt.cm.Paired)

# plot the decision function
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# create grid to evaluate model
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
Z = clf.decision_function(xy).reshape(XX.shape)

# plot decision boundary and margins
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5,
           linestyles=['--', '-', '--'])
# plot support vectors
ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=100,
           linewidth=1, facecolors='none', edgecolors='k')

plt.axis('equal')
plt.show()

**6)** Define the Gaussian classes such that the two distributions overlap. Draw an i.i.d. sample from the joint probability distribution. Apply a 5-fold Cross-Validation (for example, using the function ```GridSearchCV```) to find the optimal parameter $C∗$ to classify this new dataset using a linear kernel.

<div class='alert alert-block alert-info'>
            Code:</div>

In [ ]:
# Generate data and verify graphically that the two classes overlap
# Re-use the code from question 3 and 5

centers = [[1, 1], [-1, -1]]

# Use make_blobs to generate the two dimensions points from the two centers
X, y = make_blobs(n_samples=500, centers=centers, n_features=2 ,random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# Plot the points 
pos = np.where(y == 1)[0]
neg = np.where(y == 0)[0]
plt.scatter(X[pos,0], X[pos,1], c='r')
plt.scatter(X[neg,0], X[neg,1], c='b')
plt.axis('equal')
plt.show()

# Create a linear SVM and train it on the training data
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)
print(f'Train/Test scores: {clf.score(X_train, y_train)}/{clf.score(X_test, y_test)}')

# Plot the data
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, s=30, cmap=plt.cm.Paired)

# plot the decision function
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# create grid to evaluate model
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
Z = clf.decision_function(xy).reshape(XX.shape)

# plot decision boundary and margins
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5,
           linestyles=['--', '-', '--'])
# plot support vectors
ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=100,
           linewidth=1, facecolors='none', edgecolors='k')

plt.axis('equal')
plt.show()

In [ ]:
# Find the best working C with a 5-fold cross-validation
# Look into a bunch of values for C
parameters = {'kernel': ['linear'], 'C': list(np.logspace(-3, 3, 5))}
# Use these parameters + a SVM models with GridSearchCV (look at the documentation !)
clf = GridSearchCV(SVC(), parameters, cv=5)
clf.fit(X_train, y_train)
print(f'Best parameters: {clf.best_params_}')
print(f'Train/Test scores: {clf.score(X_train, y_train)}/{clf.score(X_test, y_test)}')

**7)** Show how tuning SVM hyperparameters on training data, for example by taking a Gaussian kernel (the parameters are therefore $\gamma$ and $C$), can lead to overfitting.

<div class='alert alert-block alert-info'>
            Code:</div>

In [ ]:
gammas = [0.1, 1, 10, 100]
# Create a gaussian svm and vary the parameter of the kernel, check the difference between training and testing scores
kernels = ['linear', 'rbf', 'sigmoid']
C = list(np.logspace(-3, 3, 5))

for kernel in kernels:
    for gamma in gammas:
        for c in C:
            clf = SVC(kernel=kernel, C=c, gamma=gamma)
            clf.fit(X_train, y_train)
            print(f'Kernel: {kernel}, Gamma: {gamma}, C: {c}, Train/Test scores: {clf.score(X_train, y_train)}/{clf.score(X_test, y_test)}')

## Non linear SVM

**8)** Define a new binary nonlinear classification problem : for instance, define one class as a Gaussian surrounded by the other chosen as a circle class, or choose the second class as a mixture of two Gaussian in such way that the separation problem is nonlinear. Generate a non-linearly separable dataset (we could for example use the function ```make_blobs``` available in ```sklearn.datasetslibrary``` ).

<div class='alert alert-block alert-info'>
            Code:</div>

In [ ]:
# Use make blobs with three centers which are aligned, for example
# Class 0 - Class 1 - Class 0 
X,y = make_circles(n_samples=500, noise=0.2, factor=0.5, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# Plot the data
pos = np.where(y == 1)[0]
neg = np.where(y == 0)[0]
plt.scatter(X[pos,0], X[pos,1], c='r')
plt.scatter(X[neg,0], X[neg,1], c='b')
plt.axis('equal')
plt.show()


**9)** Use an SVM with a linear kernel then a Gaussian (with well-adapted parameters, that you can obtain using, again, ```GridSearchCV```) then plot the decision boundaries of these algorithms on separate graphs.

<div class='alert alert-block alert-info'>
            Code:</div>

In [ ]:
# Use the code of question 6 again

# Find the best working C with a 5-fold cross-validation
# Look into a bunch of values for C
parameters = {'kernel': ['linear'], 'C': list(np.logspace(-5, 5, 5))}
# Use these parameters + a SVM models with GridSearchCV (look at the documentation !)
clf = GridSearchCV(SVC(), parameters, cv=5)
clf.fit(X_train, y_train)
bestC = clf.best_params_['C']

print(f'Best parameters: {clf.best_params_}')
print(f'Train/Test scores: {clf.score(X_train, y_train)}/{clf.score(X_test, y_test)}')

clf = SVC(kernel='linear', C=bestC)
clf.fit(X_train, y_train)

# Plot the data
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, s=30, cmap=plt.cm.Paired)

# plot the decision function
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# create grid to evaluate model
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
Z = clf.decision_function(xy).reshape(XX.shape)

# plot decision boundary and margins
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5,
           linestyles=['--', '-', '--'])
# plot support vectors
ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=100,
           linewidth=1, facecolors='none', edgecolors='k')

plt.axis('equal')
plt.title('Linear kernel')
plt.show()




##WITH GAUSSION KERNEL
# Look into a bunch of values for C
parameters = {'kernel': ['rbf'], 'C': list(np.logspace(-5, 5, 5)), 'gamma':[0.1, 1, 10, 100]}
# Use these parameters + a SVM models with GridSearchCV (look at the documentation !)
clf = GridSearchCV(SVC(), parameters, cv=5)
clf.fit(X_train, y_train)

bestC = clf.best_params_['C']
bestGamma = clf.best_params_['gamma']

print(f'Best parameters: {clf.best_params_}')
print(f'Train/Test scores: {clf.score(X_train, y_train)}/{clf.score(X_test, y_test)}')

clf = SVC(kernel='rbf', C=bestC, gamma=bestGamma)
clf.fit(X_train, y_train)

# Plot the data
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, s=30, cmap=plt.cm.Paired)

# plot the decision function
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# create grid to evaluate model
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
Z = clf.decision_function(xy).reshape(XX.shape)

# plot decision boundary and margins
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5,
           linestyles=['--', '-', '--'])
# plot support vectors
ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=100,
           linewidth=1, facecolors='none', edgecolors='k')

plt.axis('equal')
plt.title('Gaussian kernel')
plt.show()



## Learning curve

**10)** Draw the learning curve of the algorithm : with fixed hyper-parameters and a fixed test set, calculate the training and test errors by using training sub-sets of training data of various sizes (drawn randomly). For each size, repeat the experiment a large number of times to average the performance. 
Plot the train and test error based on the size of the train set subset. Estimate and display the accuracy of the Bayes predictor on the same graph. Comment.

<div class='alert alert-block alert-info'>
            Code:</div>

In [ ]:
# Total number of training samples
n_tr = len(X_train)

# Repeat the experiment for all these training sizes
sizes = np.arange(20, n_tr, 5)

# Repeat each experiment this many times 
n_m = 20  

# Store scores in these lists
scores_train = []
scores_test = []

# Main loop: varying the training size
for size in sizes:
    score_train = 0
    score_test = 0
    # Second loop: repeating the experiment for each size
    for i in range(n_m):
        # Create a SVM, keeping the same parameters
        svm = SVC(kernel='rbf', C=bestC, gamma=bestGamma)
        # For each experiment, draw a subset of the training data of the appropriate size
        idx = np.random.choice(range(n_tr), size=size)
        X_train_reduced = X_train[idx, :]
        y_train_reduced = y_train[idx]
        
        # Fit the classifier and compute the scores on training and test data
        clf.fit(X_train_reduced, y_train_reduced)
        score_train += clf.score(X_train_reduced, y_train_reduced)
        score_test += clf.score(X_test, y_test)
    # Add the average of the scores to the lists
    scores_train.append(score_train/n_m)
    scores_test.append(score_test/n_m)


# Plot the results
plt.plot(sizes, scores_train, label='Train')
plt.plot(sizes, scores_test, label='Test')
plt.xlabel('Quantity of training data')
plt.ylabel('Accuracy')

plt.legend()
plt.show()

In [ ]:
# Do a Monte-Carlo estimation of the Bayes Risk (given the gaussian distributions you used to generate data)
# This is the same computation than in question 4). 
n_mc = X.shape[0] # Repeat this n_mc times - enough to approximate 
expectation = 0
for i in range(n_mc):
    rand = np.random.choice(y) # Random binary choice: will the point be generated from the first or second gaussian ? 
    if rand == 0:
        # First case: y = 0
        x = np.random.multivariate_normal(mean=np.mean(X[pos,:], axis=0), cov=np.cov(X[pos,:].T))
    else:
        # Second case: y = 1
        x = np.random.multivariate_normal(mean=np.mean(X[neg,:], axis=0), cov=np.cov(X[neg,:].T))

    # You have to compute the conditional posterior probability of x given the 2 gaussians 
    # Use the multivariate_normal.pdf() method !    
    p1 = multivariate_normal.pdf(x, mean=np.mean(X[pos,:], axis=0), cov=np.cov(X[pos,:].T))
    p2 = multivariate_normal.pdf(x, mean=np.mean(X[neg,:], axis=0), cov=np.cov(X[neg,:].T))
    # Compute the risk from these and add it to the total
    risk = min(p1, p2)
    risk = 1 - risk if rand == 0 else risk
    expectation += min(p1/(p1+p2), p2/(p1+p2))

expectation /= n_mc
print(f'Estimated Bayes risk: {np.around(expectation, 3)}')
print(f'Estimated Bayes accuracy: {1 - np.around(expectation, 3)}')


In [ ]:
# Add this estimation to the plot of train/test error
# This is constant: you should add a horizontal line to your graph
plt.plot(sizes, scores_train, label='Train')
plt.plot(sizes, scores_test, label='Test')

plt.axhline(y=1 - expectation, color='r', linestyle='-', label='Bayes accuracy')
plt.xlabel('Quantity of training data')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

<div class='alert alert-block alert-warning'>
            Answer:</div>

The learning curves presented for the SVM classifier demonstrate a characteristic pattern, where the model initially exhibits high training accuracy with a small dataset due to overfitting, and as more data is added, the training accuracy decreases slightly while the test accuracy improves, indicating better generalization. Both accuracies converge to a plateau, suggesting that adding more data beyond this point has little benefit, possibly due to the model reaching its capacity with the given features and hyperparameters. Intriguingly, the test accuracy exceeds the Bayes accuracy estimate, which is unusual as the Bayes accuracy represents an optimal performance boundary under the assumption of correct underlying data distribution. This discrepancy could indicate an underestimation of the Bayes accuracy, possibly due to assumptions in the Monte Carlo simulation not aligning with the actual data distribution, or it may reflect an optimistic performance due to the particular test set or model overfitting.

## Error versus complexity

**11)** Add noise to the dataset by randomly modifying the labels of some training data. Then, draw the complexity curves of the algorithm : with set train and test set, draw the train and test error as a function of the complexity (i.e. as a function of the value of the hyper-parameter controlling the complexity, or the number of support vector). Comment.

<div class='alert alert-block alert-info'>
            Code:</div>

In [ ]:
# Modify the class of some data points randomly
n_noise = 100
n_tr = len(y_train)
idx = np.random.choice(range(n_tr), n_noise)
y_train[idx] = np.random.randint(0,1)

pos = np.where(y_train == 1)[0]
neg = np.where(y_train == 0)[0]

# Visualise the data
plt.scatter(X_train[pos,0], X_train[pos,1], c='r')
plt.scatter(X_train[neg,0], X_train[neg,1], c='b')
plt.axis('equal')
plt.show()

In [ ]:
# Vary the appropriate parameter and plot the training/testing results
scores_train = []
scores_test = []

for C in np.logspace(-5, 5, 5):
    clf = SVC(kernel='rbf', C=C, gamma=bestGamma)
    clf.fit(X_train, y_train)
    scores_train.append(clf.score(X_train, y_train))
    scores_test.append(clf.score(X_test, y_test))

plt.plot(np.logspace(-5, 5, 5), scores_train, label='Train')
plt.plot(np.logspace(-5, 5, 5), scores_test, label='Test')
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.xscale('log')
plt.legend()
plt.show()

<div class='alert alert-block alert-warning'>
            Answer:</div>

## Bonus : Application to face classification

Code for downloading and organizing the data:

In [ ]:
###############################################################################
#               Face Recognition Task
###############################################################################
"""
The dataset used in this example is a preprocessed excerpt
of the "Labeled Faces in the Wild", aka LFW_:

  http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz (233MB)

  _LFW: http://vis-www.cs.umass.edu/lfw/

"""

from time import time
import pylab as pl
from sklearn.datasets import fetch_lfw_people


####################################################################
# Download the data (if not already on disk); load it as numpy arrays
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4,
                              color=True, funneled=False, slice_=None,
                              download_if_missing=True)
# data_home='.'

# introspect the images arrays to find the shapes (for plotting)
images = lfw_people.images
n_samples, h, w, n_colors = images.shape

# the label to predict is the id of the person
target_names = lfw_people.target_names.tolist()

####################################################################
# Pick a pair to classify such as
names = ['Tony Blair', 'Colin Powell']
# names = ['Donald Rumsfeld', 'Colin Powell']

idx0 = (lfw_people.target == target_names.index(names[0]))
idx1 = (lfw_people.target == target_names.index(names[1]))
images = np.r_[images[idx0], images[idx1]]
n_samples = images.shape[0]
y = np.r_[np.zeros(np.sum(idx0)), np.ones(np.sum(idx1))].astype(int)

####################################################################
# Extract features

# features using only illuminations
X = (np.mean(images, axis=3)).reshape(n_samples, -1)

# # or compute features using colors (3 times more features)
# X = images.copy().reshape(n_samples, -1)

# Scale features
X -= np.mean(X, axis=0)
X /= np.std(X, axis=0)

####################################################################
# Split data into a half training and half test set
# X_train, X_test, y_train, y_test, images_train, images_test = \
#    train_test_split(X, y, images, test_size=0.5, random_state=0)
# X_train, X_test, y_train, y_test = \
#    train_test_split(X, y, test_size=0.5, random_state=0)

indices = np.random.permutation(X.shape[0])
train_idx, test_idx = indices[:int(X.shape[0] / 2)], indices[int(X.shape[0] / 2):]
X_train, X_test = X[train_idx, :], X[test_idx, :]
y_train, y_test = y[train_idx], y[test_idx]
images_train, images_test = images[train_idx, :, :, :], images[test_idx, :, :, :]

**12)** By modifying the followind code, show the influence of the regularization parameter. For example, the prediction error can be displayed as a function of $C$ on a logarithmic scale between $1e5$ and $1e-5$.

<div class='alert alert-block alert-info'>
            Code:</div>

In [ ]:
####################################################################
# Quantitative evaluation of the model quality on the test set
print("Fitting the classifier to the training set")
t0 = time()

# Add the regularization parameter and test for a range of values
c_scores = []

for c in np.logspace(np.log10(1e-5), np.log10(1e5), num=1000):
    clf = SVC(kernel='linear', C=c)
    clf = clf.fit(X_train, y_train)
    c_scores.append([clf.score(X_test, y_test), clf.score(X_train, y_train), c])
    print(f'C: {c}, Train/Test scores: {clf.score(X_train, y_train)}/{clf.score(X_test, y_test)}')


# Plot the performances
c_scores = np.array(c_scores)


plt.plot(c_scores[:,0], label='Test')
plt.plot(c_scores[:,1], label='Train')
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
print("Predicting the people names on the testing set")
t0 = time()

# Predict labels for the X_test images with the best regularization parameter you obtained
y_pred = clf.predict(X_test)

print("done in %0.3fs" % (time() - t0))
print("Chance level : %s" % max(np.mean(y), 1. - np.mean(y)))
print("Accuracy : %s" % clf.score(X_test, y_test))

In [ ]:
####################################################################
# Qualitative evaluation of the predictions using matplotlib

def plot_gallery(images, titles, n_row=3, n_col=4):
    """Helper function to plot a gallery of portraits"""
    pl.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    pl.subplots_adjust(bottom=0, left=.01, right=.99, top=.90,
                       hspace=.35)
    for i in range(n_row * n_col):
        pl.subplot(n_row, n_col, i + 1)
        pl.imshow(images[i])
        pl.title(titles[i], size=12)
        pl.xticks(())
        pl.yticks(())


def title(y_pred, y_test, names):
    pred_name = names[int(y_pred)].rsplit(' ', 1)[-1]
    true_name = names[int(y_test)].rsplit(' ', 1)[-1]
    return 'predicted: %s\ntrue:      %s' % (pred_name, true_name)

# This will just show some examples with their associated prediction - nothing to change
prediction_titles = [title(y_pred[i], y_test[i], names)
                     for i in range(y_pred.shape[0])]

plot_gallery(images_test, prediction_titles)
pl.show()

**13)** By adding nuisance variables, thus increasing the number of variables to the number of learning
points fixed, show that performance drops.

<div class='alert alert-block alert-info'>
            Code:</div>

In [ ]:
# Add a number of nuisance variable to the existing data points, by generating randomly their values

# Train SVM classifier on the original dataset
clf_original = SVC(kernel='linear', C=1)
clf_original.fit(X_train, y_train)
accuracy_original = clf_original.score(X_test, y_test)

# Generate a range og nuisance variables from 1 to 3000 in steps of 100
num_nuisance_variables = np.arange(1000, 20000, 1000)

# Add nuisance variables to the dataset
accuracies_nuisance = []

for num_nuisance in num_nuisance_variables:
     
    nuisance_var_train = np.random.normal(loc=np.random.random(), scale=2, size=(X_train.shape[0], num_nuisance))
    nuisance_var_test = np.random.normal(loc=np.random.random(), scale=2, size=(X_test.shape[0], num_nuisance))

    X_train_nuisance = np.concatenate((X_train, nuisance_var_train), axis=1)
    X_test_nuisance = np.concatenate((X_test, nuisance_var_test), axis=1)
    
    # Train SVM classifier on the dataset with added nuisance variables
    clf_nuisance = SVC(kernel='linear', C=1)
    clf_nuisance.fit(X_train_nuisance, y_train)
    accuracy_nuisance = clf_nuisance.score(X_test_nuisance, y_test)
    accuracies_nuisance.append(accuracy_nuisance)
    
    # Compare the accuracies
    performance_drop = accuracy_original - accuracy_nuisance
    print(f'Nuisance variables: {num_nuisance}, Accuracy with nuisance variables: {accuracy_nuisance}, Performance drop: {performance_drop}')


print(f'Original accuracy: {accuracy_original}')
print(f'Accuracy with nuisance variables: {accuracy_nuisance}')
print(f'Performance drop: {performance_drop}')

In [ ]:
# Plot the performances
plt.plot(num_nuisance_variables, accuracies_nuisance)

**14)** What is the effect of choosing a non-linear RBF kernel on prediction ? You will be able to improve the prediction with a reduction of dimension based on the object ```sklearn.decomposition.RandomizedPCA```.

<div class='alert alert-block alert-info'>
            Code:</div>

In [ ]:
# Apply the SVM with the chosen kernel after dimension reduction by PCA
from sklearn.decomposition import PCA

pca = PCA()
X_PCA = pca.fit_transform(X)

# Split data into training and testing sets for both X and X_PCA
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_PCA_train, X_PCA_test, y_PCA_train, y_PCA_test = train_test_split(X_PCA, y, test_size=0.2, random_state=0)

# Fit SVM classifier and evaluate the score for original data X
parameters = {'kernel': ['linear', 'rbf'], 'C': list(np.logspace(-5, 5, 5)), 'gamma':[0.1, 1, 10, 100]}
clf = GridSearchCV(SVC(), parameters, cv=5)
clf.fit(X_train, y_train)

print("Original DATA")
print("Shape of X_train: ", X_train.shape)
print("Best parameters: ", clf.best_params_)
print("Train/Test scores: ", clf.best_estimator_.score(X_train, y_train), clf.best_estimator_.score(X_test, y_test))

# Fit SVM classifier and evaluate the score for data transformed by PCA (X_PCA)
clf_PCA = GridSearchCV(SVC(), parameters, cv=5)
clf_PCA.fit(X_PCA_train, y_PCA_train)

print("PCA DATA")
print("Shape of X_PCA_train: ", X_PCA_train.shape)
print("Best parameters: ", clf_PCA.best_params_)
print("Train/Test scores: ", clf_PCA.best_estimator_.score(X_PCA_train, y_PCA_train), clf_PCA.best_estimator_.score(X_PCA_test, y_PCA_test))
